In [ ]:
import pandas as pd
import numpy as np
from m16_mlutils.pipeline import CategoryEncoder, DataFrameSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from m16_mlutils.datatools.evaluation import eval_summary

from numpy.random import seed

seed(42)

In [ ]:
training_set = pd.read_csv('data/i__training_data.csv', 
                           header=None, index_col=None,
                           names=['offer_id', 'offer_length', 'token', 'token_position',
                                  'token_pos', 'token_length', 'upper', 'n_tokens', 'real_label'])
training_set.head()
test_set = training_set[pd.isna(training_set['real_label'])]
training_set = training_set[~pd.isna(training_set['real_label'])]
print(len(training_set))
training_set.head()

## Load data transformation pipeline

In [ ]:
def get_pipeline():
    pipeline = joblib.load('data/i__pipeline.joblib') 
    return pipeline

## Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_set, training_set['real_label'])
print(len(X_train))
print(len(X_test))
X_test.head()

# Classify

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

p1 = get_pipeline()
rfc = RandomForestClassifier()
p1.steps.append(['classifier', rfc])

p1.fit(X_train, y_train)

y_pred = p1.predict(X_test)

metrics, summary, cm = eval_summary(y_test, y_pred)

print(metrics)
print()
print(summary)

In [ ]:
real_label =p1.predict(test_set)

In [ ]:
test_set.head()

In [ ]:
test_set['real_label'] = real_label

In [ ]:
test_set.head(15)